In [13]:
from importlib.metadata import version
import torch.nn as nn

print("torch version:", version("torch"))

torch version: 2.5.1


In [14]:
from torch.utils.data import Dataset

class GPTDataset(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.inputs_ids = []
        self.targets_ids = []
        
        #Tokenize the entire text
        token_ids = tokenizer.encode(txt,allowed_special={"<|endoftext|>"})
        
        #Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            
            self.inputs_ids.append(torch.tensor(input_chunk))
            self.targets_ids.append(torch.tensor(target_chunk))
    
    def __getitem__(self, idx):
        return self.inputs_ids[idx], self.targets_ids[idx]
    
    def __len__(self):
        return len(self.inputs_ids)

In [15]:
import tiktoken
from torch.utils.data import DataLoader

def create_dataloader(txt, batch_size=4, max_length=256, stride=128, shuffle=True,drop_last=True, num_workers=0):
    
    #Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    #create dataset
    dataset = GPTDataset(txt, tokenizer, max_length, stride)
    
    #create dataloader
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last, num_workers=num_workers)
    
    return dataloader

In [16]:
import torch
# Use the components that built before to implement the input embedding pipeline

#read text
with open("the-verdict.txt","r", encoding="utf-8") as f:
    raw_text = f.read()

vocab_size = 50257
output_dim = 256
# context_length: This represents the maximum sequence length that the model can process. It ensures that for any position from 0 to context_length - 1, a unique positional embedding is available.
context_length = 1024

#token_embedding_layer
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

#pos_embedding_layer
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)


#token embeddings, pos embeddings, input embeddings
max_length = 4
dataloader = create_dataloader(raw_text, batch_size=8, max_length=max_length, stride=max_length)

for batch in dataloader:
    x, y = batch
    
    # Shape: (batch_size, max_length, output_dim)
    token_embeddings = token_embedding_layer(x)
    
    # Shape: (max_length, output_dim) (broadcasted to match token_embeddings).
    pos_embeddings = pos_embedding_layer(torch.arange(max_length))
    
    input_embeddings = token_embeddings + pos_embeddings
    
    break

## Variant A: Simple implementation

In [17]:
class CausalSelfAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, qkv_bias=False):
        super().__init__()
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1))
    
    def forward(self, x):
        b, n_tokens, d_in = x.shape
        #k,q,v
        queries = self.W_query(x)
        keys = self.W_key(x)
        values = self.W_value(x)
        
        #attn_scores, attn_weights
        attn_scores = queries @ keys.transpose(1,2)
        attn_scores.masked_fill_(self.mask.bool()[:n_tokens,:n_tokens],-torch.inf)
        attn_weights = torch.softmax(attn_scores / keys.shape[-1] ** 0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)
        
        # A context vector represents the aggregated information from the sequence for a specific token.
        context_vec = attn_weights @ values
        
        return context_vec

In [18]:
class MultiHeadAttentionWrapper(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        self.heads = nn.ModuleList(
            [CausalSelfAttention(d_in, d_out, context_length, dropout, qkv_bias) for _ in range(num_heads)]
        )
        self.out_proj = nn.Linear(d_out * num_heads, d_out * num_heads)
    
    def forward(self, x):
        context_vec = torch.cat([head(x) for head in self.heads], dim=-1)
        return self.out_proj(context_vec)

In [19]:
#usage
vocab_size = 50257
output_dim = 256
max_len = 1024

context_length = max_len
d_in = output_dim

num_heads = 2
d_out = d_in // num_heads


mha = MultiHeadAttentionWrapper(d_in, d_out, context_length, 0.0, num_heads)

batch = input_embeddings
context_vecs = mha(batch)

print("context_vecs.shape:", context_vecs.shape)

context_vecs.shape: torch.Size([8, 4, 256])


## Variant B: Alternative implementation


In [20]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):   
        pass
    
    def forward(self, x):
        pass

In [21]:
#usage